In [ ]:
from warnings import filterwarnings
filterwarnings('ignore')
%matplotlib inline

In [ ]:
!git clone https://github.com/ieee8023/covid-chestxray-dataset.git
!wget https://raw.githubusercontent.com/ashvinijangid/covid-19-x-ray/master/data_tranform.py
!wget https://raw.githubusercontent.com/ashvinijangid/covid-19-x-ray/master/dataset.py
!wget https://raw.githubusercontent.com/ashvinijangid/covid-19-x-ray/master/model.py
!wget https://raw.githubusercontent.com/ashvinijangid/covid-19-x-ray/master/train.py

In [ ]:
from train import train, predict

In [ ]:
train(10, 32)

In [ ]:
predict()